In [30]:
# Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from datetime import datetime
from datetime import time
import plotly.express as px
import os
from sklearn.impute import KNNImputer

In [31]:
# Seteamos la ruta del directorio raiz

path_personal = "C:/Users/diego/OneDrive/Documentos/GitHub/Proyecto-Maestria/Floreña Ap-13/240115/"
path_halli = "x"

os.chdir(path_personal)

In [32]:
def ajuste_base(base,dia):
    
    '''
    Funcion que ajusta un archivo .csv extraidos del programa WebPlotDigitizer con las siguientes tareas.
    - Elimina NaN
    - Ajusta los valores negativos a cero
    - Convierte los segundos en tiempo a partir de una fecha
    
    Devuelve un dataframe por cada parametro    
    '''
    
    # organizamos cada variable como columnas indivivuales
    base = base.drop([0],axis=0)
    base = base.astype(float)
    Block_Height = base[["Block Height","Unnamed: 1"]].rename(columns={'Block Height': 'Time', 'Unnamed: 1': 'Block Height'}).sort_values(by=['Time'])
    Hook_Load = base[["Hook Load","Unnamed: 3"]].rename(columns={'Hook Load': 'Time', 'Unnamed: 3': 'Hook Load'}).sort_values(by=['Time'])
    WOB = base[["WOB","Unnamed: 5"]].rename(columns={'WOB': 'Time', 'Unnamed: 5': 'WOB'}).sort_values(by=['Time'])
    Torque =  base[["Torque","Unnamed: 7"]].rename(columns={'Torque': 'Time', 'Unnamed: 7': 'Torque'}).sort_values(by=['Time'])
    Flow_In =  base[["Flow In","Unnamed: 9"]].rename(columns={'Flow In': 'Time', 'Unnamed: 9': 'Flow In'}).sort_values(by=['Time'])
    Pump_Pressure =  base[["Pump Pressure","Unnamed: 11"]].rename(columns={'Pump Pressure': 'Time', 'Unnamed: 11': 'Pump Pressure'}).sort_values(by=['Time'])
    ROP =  base[["ROP","Unnamed: 13"]].rename(columns={'ROP': 'Time', 'Unnamed: 13': 'ROP'}).sort_values(by=['Time'])
    RPM = base[["RPM","Unnamed: 15"]].rename(columns={'RPM': 'Time', 'Unnamed: 15': 'RPM'}).sort_values(by=['Time'])

    # Eliminamos valores nulos
    Block_Height=Block_Height.dropna()
    Hook_Load=Hook_Load.dropna()
    WOB=WOB.dropna()
    Torque=Torque.dropna()
    Flow_In=Flow_In.dropna()
    Pump_Pressure=Pump_Pressure.dropna()
    RPM=RPM.dropna()
    ROP=ROP.dropna()

    # Eliminamos valores negativos ajustandolos a ceros
    Block_Height.loc[Block_Height["Block Height"] < 0, "Block Height"] = 0
    Block_Height.loc[Block_Height["Time"] < 0, "Time"] = 0
    Hook_Load.loc[Hook_Load["Hook Load"] < 0, "Hook Load"] = 0
    Hook_Load.loc[Hook_Load["Time"] < 0, "Time"] = 0
    WOB.loc[WOB["WOB"] < 0, "WOB"] = 0
    WOB.loc[WOB["Time"] < 0, "Time"] = 0
    Torque.loc[Torque["Torque"] < 0, "Torque"] = 0
    Torque.loc[Torque["Time"] < 0, "Time"] = 0
    Flow_In.loc[Flow_In["Flow In"] < 0, "Flow In"] = 0
    Flow_In.loc[Flow_In["Time"] < 0, "Time"] = 0
    Pump_Pressure.loc[Pump_Pressure["Pump Pressure"] < 0, "Pump Pressure"] = 0
    Pump_Pressure.loc[Pump_Pressure["Time"] < 0, "Time"] = 0
    RPM.loc[RPM["RPM"] < 0, "RPM"] = 0
    RPM.loc[RPM["Time"] < 0, "Time"] = 0
    ROP.loc[ROP["ROP"] < 0, "ROP"] = 0
    ROP.loc[ROP["Time"] < 0, "Time"] = 0
    
    # Convertimos los segundos en tiempo
    Block_Height.Time=pd.to_datetime(Block_Height.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Hook_Load.Time=pd.to_datetime(Hook_Load.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    WOB.Time=pd.to_datetime(WOB.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Torque.Time=pd.to_datetime(Torque.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Flow_In.Time=pd.to_datetime(Flow_In.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Pump_Pressure.Time=pd.to_datetime(Pump_Pressure.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    RPM.Time=pd.to_datetime(RPM.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    ROP.Time=pd.to_datetime(ROP.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    
    return(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)

In [33]:
def Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP):
    
    '''
    Funcion que realiza las siguientes tareas:
    - Organiza por tiempo los datos
    - Convierte las columnas de tiempo en indice de tipo datetimeindex
    - Realiza un resampling para cada 3 segundos.
    - Organiza en un solo dataframe todas las variables con el mismo indice
    - Si despues del resampling hay valores nulos, los imputa con KNN
    
    Devuelve un dataframe merged, resampled e imputado    
    '''
    
    Block_Height.sort_values(by=['Time'])
    Hook_Load.sort_values(by=['Time'])
    WOB.sort_values(by=['Time'])
    Torque.sort_values(by=['Time'])
    Flow_In.sort_values(by=['Time'])
    Pump_Pressure.sort_values(by=['Time'])
    RPM.sort_values(by=['Time'])
    ROP.sort_values(by=['Time'])
    
    Block_Height['Time'] = pd.to_datetime(Block_Height['Time'])
    Block_Height.set_index('Time',inplace=True)
    Hook_Load['Time'] = pd.to_datetime(Hook_Load['Time'])
    Hook_Load.set_index('Time',inplace=True)
    WOB['Time'] = pd.to_datetime(WOB['Time'])
    WOB.set_index('Time',inplace=True)
    Torque['Time'] = pd.to_datetime(Torque['Time'])
    Torque.set_index('Time',inplace=True)
    Flow_In['Time'] = pd.to_datetime(Flow_In['Time'])
    Flow_In.set_index('Time',inplace=True)
    Pump_Pressure['Time'] = pd.to_datetime(Pump_Pressure['Time'])
    Pump_Pressure.set_index('Time',inplace=True)
    RPM['Time'] = pd.to_datetime(RPM['Time'])
    RPM.set_index('Time',inplace=True)
    ROP['Time'] = pd.to_datetime(ROP['Time'])
    ROP.set_index('Time',inplace=True)
    
    Block_Height=Block_Height.resample("3S").mean()
    Hook_Load=Hook_Load.resample("3S").mean()
    WOB=WOB.resample("3S").mean()
    Torque=Torque.resample("3S").mean()
    Flow_In=Flow_In.resample("3S").mean()
    Pump_Pressure=Pump_Pressure.resample("3S").mean()
    RPM=RPM.resample("3S").mean()
    ROP=ROP.resample("3S").mean()
        
    Tabla=pd.concat([Block_Height, Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP], axis=1)
    
    imputer = KNNImputer(n_neighbors=5,weights="uniform")
    df_filled = imputer.fit_transform(Tabla)
    
    Tabla_Filled=pd.DataFrame(df_filled).merge(pd.DataFrame(Tabla.index), left_index=True, right_index=True)
    Tabla_Filled.set_index('Time',inplace=True)
    Tabla_Filled.columns=Tabla.columns
    Tabla_Filled = Tabla_Filled.rolling(window=20).mean()
    
    return(Tabla_Filled)

In [34]:
def unir_data(df_old,df_new):
    '''
    Esta funcion une el datframe de los datos previos con el dataframe de los datos nuevos
    '''
    
    Data_Export = pd.concat([df_old, df_new], axis=1).groupby(axis=1, level=0).mean()
    return Data_Export

In [35]:
def plot_incident_df(df,size):
    '''

    Funcion que toma un dataframe con indice datetime y con columnas de cada variable y realiza las siguientes tareas:
    
    - Grafica el incident

    '''

    fig1, ax1 = plt.subplots(nrows=1, ncols=3, figsize=(12,size), sharey=True)
    fig1.suptitle("Incident Log", fontsize=22)
    fig1.subplots_adjust(top=0.85,wspace=0.2)
 
    #1st track: Block Height, Hook Load & ROP
    ax01=ax1[0].twiny()
    ax01.invert_xaxis()
    ax01.invert_yaxis()
    ax01.set_xlim(0,120)
    ax01.set_ylim([np.max(df.index), np.min(df.index)])
    ax01.spines['top'].set_position(('outward',0))
    ax01.set_xlabel("Block Height [ft]")
    ax01.plot(df["Block Height"], df.index,"--", label='Block Height [ft]', color='red')
    ax01.set_xlabel('Block Height [ft]',color='red')    
    ax01.tick_params(axis='x', colors='red')
    
    ax11=ax1[0].twiny()
    ax11.invert_xaxis()
    ax11.invert_yaxis()
    ax11.set_xlim(0,900)
    ax11.plot(df["Hook Load"], df.index, label='Hook Load [Klb]', color='blue') 
    ax11.spines['top'].set_position(('outward',40))
    ax11.set_xlabel('Hook Load [Klb]',color='blue')    
    ax11.tick_params(axis='x', colors='blue')

    ax21=ax1[0].twiny() 
    ax21.invert_xaxis()
    ax21.invert_yaxis()
    ax21.invert_yaxis()
    ax21.set_xlim(0,60)
    ax21.plot(df["ROP"], df.index, label='ROP[ft/hr]', color='magenta') 
    ax21.spines['top'].set_position(('outward',80))
    ax21.set_xlabel('ROP[ft/hr]',color='magenta')    
    ax21.tick_params(axis='x', colors='magenta')
    ax21.grid(True)
    
    #2nd track: WOB, Torque, RPM
    ax02=ax1[1].twiny()
    ax02.invert_xaxis()
    ax02.invert_yaxis()
    ax02.set_xlim(0,90)
    ax02.spines['top'].set_position(('outward',0))
    ax02.set_xlabel('WOB[KLb]', color='lime')
    ax02.plot(df.WOB, df.index,"--", label='WOB[KLb]', color='lime')
    ax02.tick_params(axis='x', colors='lime')    
    ax02.grid(True)
    
    ax12=ax1[1].twiny()
    ax12.invert_xaxis()
    ax12.invert_yaxis()
    ax12.set_xlim(0,36)
    ax12.plot(df.Torque, df.index,":", label='Torque[KLb-ft]', color='blue') 
    ax12.spines['top'].set_position(('outward',40))
    ax12.set_xlabel('Torque[KLb-ft]', color='blue')    
    ax12.tick_params(axis='x', colors='blue')
    
    ax22=ax1[1].twiny()
    ax22.invert_xaxis()
    ax22.invert_yaxis()
    ax22.set_xlim(0,700)
    ax22.plot(df["RPM"], df.index, '-',label='RPM[Rev/min]', color='dimgrey') 
    ax22.spines['top'].set_position(('outward',80))
    ax22.set_xlabel('RPM[Rev/min]',color='dimgrey')
    ax22.tick_params(axis='x', colors='dimgrey')
    
    #3rd track: Flow In & Pump Pressure
    ax03=ax1[2].twiny()
    ax03.invert_xaxis()
    ax03.invert_yaxis()
    ax03.set_xlim(0,1200)
    ax03.spines['top'].set_position(('outward',0))
    ax03.set_xlabel('Flow In[GPM]')
    ax03.plot(df["Flow In"], df.index,'--', label='DT[us/ft]', color='red')
    ax03.set_xlabel('Flow In[GPM]', color='red')    
    ax03.tick_params(axis='x', colors='red')
    ax03.grid(True)
    
    ax13=ax1[2].twiny()
    ax13.invert_xaxis()
    ax13.invert_yaxis()
    ax13.set_xlim(0,6000)
    ax13.plot(df["Pump Pressure"], df.index, label='Pump Pressure[psi]', color='blue') 
    ax13.spines['top'].set_position(('outward',40))
    ax13.set_xlabel('Pump Pressure[psi]', color='blue')    
    ax13.tick_params(axis='x', colors='blue')
    
    return(fig1)

In [36]:
def plot_incident_stuck(df,size):
    '''

    Funcion que toma un dataframe con indice datetime y con columnas de cada variable y realiza las siguientes tareas:
    
    - Grafica el incident

    '''
    
    %matplotlib widget
    
    fig2, ax1 = plt.subplots(nrows=1, ncols=4, figsize=(10,size), sharey=True)
    fig2.suptitle("Incident Log", fontsize=22)
    fig2.subplots_adjust(top=0.85,wspace=0.2)
 
    #1st track: Block Height, Hook Load & ROP
    ax01=ax1[0].twiny()
    ax01.invert_xaxis()
    ax01.invert_yaxis()
    ax01.set_xlim(0,120)
    ax01.set_ylim([np.max(df.index), np.min(df.index)])
    ax01.spines['top'].set_position(('outward',0))
    ax01.set_xlabel("Block Height [ft]")
    ax01.plot(df["Block Height"], df.index,"--", label='Block Height [ft]', color='red')
    ax01.set_xlabel('Block Height [ft]',color='red')    
    ax01.tick_params(axis='x', colors='red')
    
    ax11=ax1[0].twiny()
    ax11.invert_xaxis()
    ax11.invert_yaxis()
    ax11.set_xlim(0,900)
    ax11.plot(df["Hook Load"], df.index, label='Hook Load [Klb]', color='blue') 
    ax11.spines['top'].set_position(('outward',40))
    ax11.set_xlabel('Hook Load [Klb]',color='blue')    
    ax11.tick_params(axis='x', colors='blue')

    ax21=ax1[0].twiny() 
    ax21.invert_xaxis()
    ax21.invert_yaxis()
    ax21.invert_yaxis()
    ax21.set_xlim(0,150)
    ax21.plot(df["ROP"], df.index, label='ROP[ft/hr]', color='magenta') 
    ax21.spines['top'].set_position(('outward',80))
    ax21.set_xlabel('ROP[ft/hr]',color='magenta')    
    ax21.tick_params(axis='x', colors='magenta')
    ax21.grid(True)
    
    #2nd track: WOB, Torque, RPM
    ax02=ax1[1].twiny()
    ax02.invert_xaxis()
    ax02.invert_yaxis()
    ax02.set_xlim(0,90)
    ax02.spines['top'].set_position(('outward',0))
    ax02.set_xlabel('WOB[KLb]', color='lime')
    ax02.plot(df.WOB, df.index,"--", label='WOB[KLb]', color='lime')
    ax02.tick_params(axis='x', colors='lime')    
    ax02.grid(True)
    
    ax12=ax1[1].twiny()
    ax12.invert_xaxis()
    ax12.invert_yaxis()
    ax12.set_xlim(0,36)
    ax12.plot(df.Torque, df.index,":", label='Torque[KLb-ft]', color='blue') 
    ax12.spines['top'].set_position(('outward',40))
    ax12.set_xlabel('Torque[KLb-ft]', color='blue')    
    ax12.tick_params(axis='x', colors='blue')
    
    ax22=ax1[1].twiny()
    ax22.invert_xaxis()
    ax22.invert_yaxis()
    ax22.set_xlim(0,180)
    ax22.plot(df["RPM"], df.index, '-',label='RPM[Rev/min]', color='dimgrey') 
    ax22.spines['top'].set_position(('outward',80))
    ax22.set_xlabel('RPM[Rev/min]',color='dimgrey')
    ax22.tick_params(axis='x', colors='dimgrey')
    
    #3rd track: Flow In & Pump Pressure
    ax03=ax1[2].twiny()
    ax03.invert_xaxis()
    ax03.invert_yaxis()
    ax03.set_xlim(0,1200)
    ax03.spines['top'].set_position(('outward',0))
    ax03.set_xlabel('Flow In[GPM]')
    ax03.plot(df["Flow In"], df.index,'--', label='DT[us/ft]', color='red')
    ax03.set_xlabel('Flow In[GPM]', color='red')    
    ax03.tick_params(axis='x', colors='red')
    ax03.grid(True)
    
    ax13=ax1[2].twiny()
    ax13.invert_xaxis()
    ax13.invert_yaxis()
    ax13.set_xlim(0,6000)
    ax13.plot(df["Pump Pressure"], df.index, label='Pump Pressure[psi]', color='blue') 
    ax13.spines['top'].set_position(('outward',40))
    ax13.set_xlabel('Pump Pressure[psi]', color='blue')    
    ax13.tick_params(axis='x', colors='blue')
    
    #4rd track: Stuck Pipe & Stuck Pipe Proba
    ax04=ax1[3].twiny()
    #ax04.invert_xaxis()
    #ax04.invert_yaxis()
    ax04.set_xlim(0,1)
    ax04.spines['top'].set_position(('outward',0))
    ax04.set_xlabel('Stuck Pipe')
    ax04.plot(df["Pega"], df.index, label="Tuberia Libre o Pegada", color="red")
    #ax04.fill_between(df.Pega, df.index, label="Tuberia Libre o Pegada", color='red',alpha=0.5) 
    ax04.set_xlabel('Libre [0] - Pegado [1]', color='blue')    
    ax04.tick_params(axis='x', colors='blue')
    ax04.grid(True)
    
    return(fig2)

# Cargamos los datos de todos los archivos

In [37]:
base1 = pd.read_csv("wpd_datasets.csv")

dia = "2014-01-15"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base1,dia)
Tabla1=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)


C:\Users\diego\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [38]:
Tabla1

,Block Height,Hook Load,WOB,Torque,Flow In,Pump Pressure,RPM,ROP
Time,,,,,,,,
2014-01-15 08:56:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-15 08:56:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-15 08:56:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-15 08:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-15 08:57:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2014-01-15 19:01:24,51.760459,252.526227,5.913961,1.778088,327.130391,614.625472,98.972816,0.0
2014-01-15 19:01:27,53.142393,252.522671,5.913961,1.897344,361.320660,700.704199,99.429731,0.0
2014-01-15 19:01:30,55.788037,252.536651,5.876830,2.100223,390.336769,797.268208,99.783493,0.0


In [39]:
p1=plot_incident_df(Tabla1,10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
# Ajustes de valores anomalos

Tabla1 = Tabla1.dropna()
Tabla1.loc[Tabla1.RPM==0,"Torque"]=0
Tabla1.loc[Tabla1["Pump Pressure"]==0,"Flow In"]=0
Tabla1["Evento"] = "Evento 5"

Tabla1["Pega"]=0
Tabla1.loc[Tabla1.index > "2014-01-15 17:02:00","Pega"]=1

C:\Users\diego\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\diego\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\diego\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [41]:
# Creacion de archivo procesado

path_personal = "C:/Users/diego/OneDrive/Documentos/GitHub/Proyecto-Maestria/Data Final/Evento 5.csv"
path_halli = "x"

Tabla1.to_csv(path_or_buf=path_personal,index=True,header=True)

In [42]:
p1=plot_incident_stuck(Tabla1,10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …